In [1]:
import pandas as pd
import sqlite3

# Membuat koneksi ke database SQLite
conn = sqlite3.connect('customer_segmentation.db')

In [2]:
lists = pd.read_csv('https://raw.githubusercontent.com/imomenujung/SQL-Project/main/Data-Analyst/Customer-lifetime-value/Data/List%20of%20Orders.csv')
details = pd.read_csv('https://raw.githubusercontent.com/imomenujung/SQL-Project/main/Data-Analyst/Customer-lifetime-value/Data/Order%20Details.csv')
target = pd.read_csv('https://raw.githubusercontent.com/imomenujung/SQL-Project/main/Data-Analyst/Customer-lifetime-value/Data/Sales%20target.csv')

In [4]:
lists.head()

,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


In [5]:
details.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones


In [6]:
target.head()

,Month of Order Date,Category,Target
0,Apr-18,Furniture,10400.0
1,May-18,Furniture,10500.0
2,Jun-18,Furniture,10600.0
3,Jul-18,Furniture,10800.0
4,Aug-18,Furniture,10900.0


In [3]:
#Create the database conection
connection = sqlite3.connect('bike_store.db')

# Insert data into database
lists.to_sql('lists', conn, if_exists='replace', index=False)
details.to_sql('details', conn, if_exists='replace', index=False)
target.to_sql('target', conn, if_exists='replace', index=False)

36

In [9]:
query = """
SELECT
    CustomerName,
    COUNT("Order ID") AS NumberOfTransactions
FROM
    lists
GROUP BY
    CustomerName
    ORDER BY
    NumberOfTransactions;
"""

df = pd.read_sql_query(query, conn)
df

,CustomerName,NumberOfTransactions
0,None,0
1,Aakanksha,1
2,Aashna,1
3,Aastha,1
4,Aayush,1
...,...,...
328,Yogesh,4
329,Abhishek,5
330,Pooja,5
331,Shubham,5


In [10]:
query = """
SELECT
    l.CustomerName,
    SUM(d.Quantity * d.Amount) as TotalAmountSpent
FROM
    lists l
JOIN
    details d ON l."Order ID" = d."Order ID"
GROUP BY
    l.CustomerName
ORDER BY
    TotalAmountSpent DESC;
"""

df = pd.read_sql_query(query, conn)
df


,CustomerName,TotalAmountSpent
0,Yaanvi,103435.0
1,Seema,64222.0
2,Soumya,46086.0
3,Pooja,43222.0
4,Shishu,38896.0
...,...,...
327,Vipul,16.0
328,Ananya,16.0
329,Rohit,15.0
330,Hemangi,13.0


In [11]:
query = """
SELECT
    l.CustomerName,
    AVG(d.Quantity * d.Amount) as AvgPurchaseValue
FROM
    lists l
JOIN
    details d ON l."Order ID" = d."Order ID"
GROUP BY
    l.CustomerName
ORDER BY
    AvgPurchaseValue DESC;
"""

df = pd.read_sql_query(query, conn)
df


,CustomerName,AvgPurchaseValue
0,Seema,12844.400000
1,Arti,11682.000000
2,Shreyoshe,9492.000000
3,Pranjali,8750.000000
4,Yaanvi,7956.538462
...,...,...
327,Vipul,16.000000
328,Ananya,16.000000
329,Rohit,15.000000
330,Hemangi,13.000000


In [14]:
query = """
SELECT
    CustomerName,
    COUNT(DISTINCT "Order Date") AS PurchaseFrequency
FROM
    lists
GROUP BY
    CustomerName
ORDER BY
    PurchaseFrequency DESC;
"""

df = pd.read_sql_query(query, conn)
df

,CustomerName,PurchaseFrequency
0,Shreya,6
1,Shubham,5
2,Pooja,5
3,Abhishek,5
4,Yogesh,4
...,...,...
328,Aayush,1
329,Aastha,1
330,Aashna,1
331,Aakanksha,1


In [16]:
import pandas as pd
query = """
WITH purchase_frequency AS (
    SELECT
        CustomerName,
        COUNT(DISTINCT "Order Date") AS PurchaseFrequency
    FROM
        lists
    GROUP BY
        CustomerName
)
SELECT
    CustomerName,
    AVG(PurchaseFrequency) AS AveragePurchaseFrequency
FROM
    purchase_frequency
GROUP BY
    CustomerName;

"""

df = pd.read_sql_query(query, conn)
df


,CustomerName,AveragePurchaseFrequency
0,None,0.0
1,Aakanksha,1.0
2,Aarushi,3.0
3,Aashna,1.0
4,Aastha,1.0
...,...,...
328,Wale,1.0
329,Yaanvi,2.0
330,Yash,1.0
331,Yogesh,4.0


In [17]:
import pandas as pd
query = """
WITH customer_transactions AS (
    SELECT
        l.CustomerName,
        d.Quantity * d.Amount as TransactionAmount,
        l."Order Date" as TransactionDate
    FROM
        lists l
    JOIN
        details d ON l."Order ID" = d."Order ID"
),
clv_calculations AS (
    SELECT
        CustomerName,
        SUM(TransactionAmount) AS TotalSpent,
        COUNT(DISTINCT TransactionDate) AS PurchaseFrequency,
        AVG(TransactionAmount) AS AveragePurchaseValue
    FROM
        customer_transactions
    GROUP BY
        CustomerName
)
SELECT
    CustomerName,
    (TotalSpent * 0.30) AS CLV -- Assuming 30% gross margin
FROM
    clv_calculations;
"""

df = pd.read_sql_query(query, conn)
df


,CustomerName,CLV
0,Aakanksha,81.6
1,Aarushi,6613.5
2,Aashna,3050.7
3,Aastha,4323.3
4,Aayush,505.8
...,...,...
327,Wale,93.0
328,Yaanvi,31030.5
329,Yash,2201.1
330,Yogesh,8838.0
